## Cosinüs 

In [ ]:
import numpy as np
from scipy import io
import warnings
from sklearn.metrics.pairwise import pairwise_distances
warnings.filterwarnings("ignore")

# work .mat file
mat = io.loadmat('matrix.mat')
matrix=mat["matrix"]
sim_list=[]
pre_list=[]
sim_list.append(matrix[0][0])
sim_list.append(matrix[13][6])
matrix[0][0]=0

def user_rating_average(mat):
    sum1=0
    average_list=[]
    for x in range(len(mat)):
        for i in mat[x]:
            sum1=sum1+i
        sum1=sum1/len(mat[x].nonzero()[0])
        average_list.append(sum1)
        sum1=0
    return average_list

user_average_list=user_rating_average(matrix)
user_average_list=np.nan_to_num(user_average_list, nan=0)

user_similarity_list = pairwise_distances(matrix, metric='cosine')
similar_n = user_similarity_list[0].argsort()

def user_rating_pred(avr_list,mat_row,user_1_rating,pre_id):
    result=0
    pay=0
    for usr in range(len(avr_list)):
        if usr == pre_id:
            continue
        else:
            result+=mat_row[usr]*(user_1_rating[usr][pre_id]-avr_list[usr])
            pay+=mat_row[usr]
    return avr_list[pre_id]+(result/pay)
# tüm oy vermeyenleri çıkart
# sim ilk hesapla
telsim=user_rating_pred(user_average_list,user_similarity_list[0],matrix,similar_n[0])
matrix[13][6]=0
similar_n1 = user_similarity_list[13].argsort()
telsim1=user_rating_pred(user_average_list,user_similarity_list[13],matrix,similar_n1[0])
pre_list.append(telsim)
pre_list.append(telsim1)
print("User-{} prediction rating:".format(similar_n[0]),telsim)
print("User-{} prediction rating:".format(similar_n1[0]),telsim1)

def calulate_error_avg(list1,list2):
    total=0
    for i in range(len(list1)):
        total=total+(list1[i]-list2[i])
    return total/len(list1)
a=calulate_error_avg(sim_list,pre_list)
print("Error:",a)

In [9]:
import numpy as np
from scipy import io
import warnings
from sklearn.metrics.pairwise import pairwise_distances
warnings.filterwarnings("ignore")

mat = io.loadmat('matrix.mat')
matrix=mat["matrix"]
user_similarity_list = pairwise_distances(matrix, metric='cosine')
similar_n = user_similarity_list[13].argsort()


In [10]:
similar_n

array([13,  5,  0, 17, 14, 12,  9, 15, 16, 10,  6,  1,  7,  8, 19, 11, 18,
        4,  3,  2], dtype=int64)

In [11]:

user_similarity_list = pairwise_distances(matrix, metric='cosine')
similar_n = user_similarity_list[0].argsort()

In [12]:
similar_n

array([ 0, 17, 13,  5, 12,  9, 14, 15,  1, 16,  6, 10, 19,  8,  4, 11,  7,
       18,  3,  2], dtype=int64)

In [13]:
similar_n

array([ 0, 17, 13,  5, 12,  9, 14, 15,  1, 16,  6, 10, 19,  8,  4, 11,  7,
       18,  3,  2], dtype=int64)

# Pearson

In [14]:
import numpy as np
from scipy import stats, io
import warnings
warnings.filterwarnings("ignore")

# work .mat file
mat = io.loadmat('matrix.mat')
matrix=mat["matrix"]
sim_list=[]
pre_list=[]
sim_list.append(matrix[0][6])
sim_list.append(matrix[13][6])
matrix[0][0]=0


def user_rating_average(mat):
    sum1=0
    average_list=[]
    for x in range(len(mat)):
        for i in mat[x]:
            sum1=sum1+i
        sum1=sum1/len(mat[x].nonzero()[0])
        average_list.append(sum1)
        sum1=0
    return average_list

user_average_list=user_rating_average(matrix)
user_average_list=np.nan_to_num(user_average_list, nan=0)

def pearson_similarity(mat):
    arr=np.zeros((20, 20))
    for i in range(len(mat)):
        for j in range(len(mat)):
            pearson=stats.pearsonr(mat[i],mat[j])
            arr[i][j]=pearson[0]
    return arr

user_similarity_list=pearson_similarity(matrix)

user_similarity_list=np.nan_to_num(user_similarity_list, nan=0)

similar_n = user_similarity_list[0].argsort()

def cal_pay(avr_list,mat_row,user_1_rating,pre_id):
    result=0
    pay=0
    for item in range(len(avr_list)):
        if item == pre_id:
            continue
        else:
            result+=mat_row[item]*(user_1_rating[item][pre_id]-avr_list[item])
            pay+=mat_row[item]
    return avr_list[pre_id]+(result/pay)

# tüm oy vermeyenleri çıkart
# sim ilk hesapla
telsim=cal_pay(user_average_list,user_similarity_list[0],matrix,similar_n[-1])
matrix[13][6]=0
similar_n1 = user_similarity_list[13].argsort()
telsim1=cal_pay(user_average_list,user_similarity_list[13],matrix,similar_n1[-1])
pre_list.append(telsim)
pre_list.append(telsim1)
print("User-{} prediction rating:".format(similar_n[-1]),telsim)
print("User-{} prediction rating:".format(similar_n1[-1]),telsim1)
def calulate_error_avg(list1,list2):
    total=0
    for i in range(len(list1)):
        total=total+(list1[i]-list2[i])
    return total/len(list1)
a=calulate_error_avg(sim_list,pre_list)
print("Error:",a)

User-0 prediction rating: 4.077400815551107
User-13 prediction rating: 3.944898740513024
Error: 0.4888502219679347


In [15]:
sim_list

[4, 5]

In [16]:
pre_list

[4.077400815551107, 3.944898740513024]

# ml-100k dataset

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import math

In [4]:
rs_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('Dataset/ml-100k/ua.base', sep='\t', names=rs_cols, encoding='latin-1')
ratings_test = pd.read_csv('Dataset/ml-100k/ua.base', sep='\t', names=rs_cols, encoding='latin-1')

In [5]:
n_users_base = ratings_base['user_id'].unique().max()
n_items_base = ratings_base['movie_id'].unique().max()
n_users_base,n_items_base

(943, 1682)

In [6]:
n_users_test = ratings_test['user_id'].unique().max()
n_items_test = ratings_test['movie_id'].unique().max()
n_users_test,n_items_test

(943, 1682)

In [7]:
train_matrix = np.zeros((n_users_base, n_items_base))
test_matrix = np.zeros((n_users_test, n_items_test))

In [8]:
for line in ratings_base.itertuples():
    train_matrix[line[1]-1,line[2]-1] = line[3]

In [9]:
test_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
train_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [11]:
n_users_base

943

In [12]:
random_users_id_list=[]
while True:
    rnd=np.random.randint(943)
    if len(random_users_id_list) < 94:
        if rnd in random_users_id_list:
            continue
        else:
            random_users_id_list.append(rnd)
    else:
        break

In [13]:
random_users_id_list.sort()

In [14]:
print(random_users_id_list)

[0, 7, 16, 22, 29, 43, 56, 67, 69, 82, 83, 85, 104, 110, 142, 164, 166, 176, 188, 214, 216, 221, 236, 237, 244, 264, 270, 272, 291, 292, 297, 301, 303, 324, 331, 336, 353, 361, 363, 373, 387, 389, 397, 408, 410, 419, 420, 428, 441, 459, 488, 529, 537, 539, 546, 553, 559, 567, 571, 574, 578, 583, 610, 616, 640, 641, 650, 674, 694, 711, 713, 714, 723, 726, 743, 766, 781, 783, 788, 823, 842, 845, 852, 858, 859, 870, 877, 880, 899, 914, 920, 927, 937, 940]


## Finding and keeping the first items and ratings that 10% users voted for 

In [15]:
users_first_rating_movie=[]

In [16]:
for user in random_users_id_list:
    for item in range(n_items_base):
        if train_matrix[user][item] == 0:
            continue
        else:
            users_first_rating_movie.append(item)
            break

In [17]:
print(random_users_id_list)

[0, 7, 16, 22, 29, 43, 56, 67, 69, 82, 83, 85, 104, 110, 142, 164, 166, 176, 188, 214, 216, 221, 236, 237, 244, 264, 270, 272, 291, 292, 297, 301, 303, 324, 331, 336, 353, 361, 363, 373, 387, 389, 397, 408, 410, 419, 420, 428, 441, 459, 488, 529, 537, 539, 546, 553, 559, 567, 571, 574, 578, 583, 610, 616, 640, 641, 650, 674, 694, 711, 713, 714, 723, 726, 743, 766, 781, 783, 788, 823, 842, 845, 852, 858, 859, 870, 877, 880, 899, 914, 920, 927, 937, 940]


In [18]:
print(users_first_rating_movie)

[0, 6, 6, 0, 1, 0, 0, 8, 0, 0, 0, 241, 263, 241, 257, 14, 7, 0, 0, 7, 10, 0, 8, 120, 132, 0, 0, 267, 0, 0, 0, 265, 110, 0, 0, 14, 6, 244, 260, 0, 0, 0, 0, 5, 0, 13, 3, 0, 1, 0, 242, 55, 3, 8, 268, 0, 0, 5, 8, 78, 3, 24, 261, 16, 22, 0, 126, 85, 241, 3, 0, 0, 241, 0, 0, 0, 49, 257, 0, 242, 0, 1, 257, 2, 25, 3, 7, 0, 8, 257, 14, 8, 0, 0]


In [19]:
for i in range(94):
    print(train_matrix[random_users_id_list[i]][users_first_rating_movie[i]], end=", ")

5.0, 3.0, 4.0, 5.0, 3.0, 4.0, 5.0, 4.0, 4.0, 4.0, 2.0, 4.0, 2.0, 4.0, 3.0, 5.0, 5.0, 3.0, 5.0, 2.0, 4.0, 4.0, 4.0, 4.0, 2.0, 5.0, 3.0, 5.0, 4.0, 2.0, 5.0, 2.0, 3.0, 2.0, 4.0, 5.0, 4.0, 4.0, 2.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 3.0, 5.0, 3.0, 3.0, 4.0, 3.0, 5.0, 5.0, 2.0, 3.0, 4.0, 1.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 1.0, 3.0, 4.0, 5.0, 3.0, 5.0, 3.0, 1.0, 3.0, 5.0, 3.0, 5.0, 3.0, 3.0, 3.0, 4.0, 2.0, 2.0, 4.0, 5.0, 4.0, 5.0, 

## create a test matrix 

In [20]:
for i in range(len(random_users_id_list)):
    test_matrix[random_users_id_list[i]][users_first_rating_movie[i]]=train_matrix[random_users_id_list[i]][users_first_rating_movie[i]]
    train_matrix[random_users_id_list[i]][users_first_rating_movie[i]]=0

In [21]:
print(random_users_id_list)

[0, 7, 16, 22, 29, 43, 56, 67, 69, 82, 83, 85, 104, 110, 142, 164, 166, 176, 188, 214, 216, 221, 236, 237, 244, 264, 270, 272, 291, 292, 297, 301, 303, 324, 331, 336, 353, 361, 363, 373, 387, 389, 397, 408, 410, 419, 420, 428, 441, 459, 488, 529, 537, 539, 546, 553, 559, 567, 571, 574, 578, 583, 610, 616, 640, 641, 650, 674, 694, 711, 713, 714, 723, 726, 743, 766, 781, 783, 788, 823, 842, 845, 852, 858, 859, 870, 877, 880, 899, 914, 920, 927, 937, 940]


In [22]:
for i in random_users_id_list:
    print(np.sum(test_matrix[i]), end=", ")

5.0, 3.0, 4.0, 5.0, 3.0, 4.0, 5.0, 4.0, 4.0, 4.0, 2.0, 4.0, 2.0, 4.0, 3.0, 5.0, 5.0, 3.0, 5.0, 2.0, 4.0, 4.0, 4.0, 4.0, 2.0, 5.0, 3.0, 5.0, 4.0, 2.0, 5.0, 2.0, 3.0, 2.0, 4.0, 5.0, 4.0, 4.0, 2.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 3.0, 5.0, 3.0, 3.0, 4.0, 3.0, 5.0, 5.0, 2.0, 3.0, 4.0, 1.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 1.0, 3.0, 4.0, 5.0, 3.0, 5.0, 3.0, 1.0, 3.0, 5.0, 3.0, 5.0, 3.0, 3.0, 3.0, 4.0, 2.0, 2.0, 4.0, 5.0, 4.0, 5.0, 

In [23]:
for i in range(94):
    print(train_matrix[random_users_id_list[i]][users_first_rating_movie[i]], end=", ")

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## creates test matrix and train matrix 

In [24]:
user_similarity = pairwise_distances(train_matrix, metric='correlation')
user_similarity=1-user_similarity
print('shape: ',user_similarity.shape)
print('type: ',type(user_similarity))
user_similarity

shape:  (943, 943)
type:  <class 'numpy.ndarray'>


array([[ 1.        ,  0.07927708, -0.00550302, ..., -0.02031016,
         0.10906835,  0.30913269],
       [ 0.07927708,  1.        ,  0.10288466, ...,  0.11963309,
         0.1442557 ,  0.03401889],
       [-0.00550302,  0.10288466,  1.        , ...,  0.01858789,
         0.09786267, -0.01432321],
       ...,
       [-0.02031016,  0.11963309,  0.01858789, ...,  1.        ,
         0.02799815, -0.00191   ],
       [ 0.10906835,  0.1442557 ,  0.09786267, ...,  0.02799815,
         1.        ,  0.09312477],
       [ 0.30913269,  0.03401889, -0.01432321, ..., -0.00191   ,
         0.09312477,  1.        ]])

## make the diogones zero 

In [25]:
len(user_similarity)

943

In [26]:
for i in range(len(user_similarity)):
    for j in range(len(user_similarity)):
        if i == j:
            user_similarity[i][j]=0
        else:
            continue

In [27]:
for i in range(len(user_similarity)):
    print(user_similarity[i][i], end=" ,")

0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,

In [28]:
def predict_user_user(train_matrix, user_similarity, n_similar = 10):
    similar_n = user_similarity.argsort()[:,n_similar:][:,::-1]# kontrol et
    pred = np.zeros((n_users_base,n_items_base))
    
    for i,users in enumerate(similar_n):
        similar_users_indexes = users
        similarity_n = user_similarity[i,similar_users_indexes]
        matrix_n = train_matrix[similar_users_indexes,:]
        rated_items = similarity_n[:,np.newaxis].T.dot(matrix_n - matrix_n.mean(axis=1)[:,np.newaxis])/ similarity_n.sum()
        pred[i,:]  = rated_items
        
    return pred

In [29]:
predictions = predict_user_user(train_matrix,user_similarity, 10) + train_matrix.mean(axis=1)[:, np.newaxis]# 30 olması daha iyi
print('predictions shape ',predictions.shape)
predictions

predictions shape  (943, 1682)


array([[ 2.30058124,  0.99016878,  0.68629725, ...,  0.24816284,
         0.25609131,  0.25760888],
       [ 1.40653539,  0.17837178,  0.21846442, ..., -0.04708573,
        -0.04961555, -0.05147659],
       [ 0.74249652,  0.09767795,  0.13221405, ..., -0.04064267,
        -0.05116303, -0.05416094],
       ...,
       [ 1.43997849,  0.19879184,  0.23292258, ..., -0.15166184,
        -0.15258599, -0.15116336],
       [ 1.57940795,  0.45416652,  0.18813963, ..., -0.08065315,
        -0.07752764, -0.08067586],
       [ 2.1705989 ,  0.87141644,  0.50317569, ..., -0.00543864,
         0.0034516 ,  0.00249059]])

In [30]:
predicted_ratings = predictions[test_matrix.nonzero()]
test_truth = test_matrix[test_matrix.nonzero()]

In [31]:
math.sqrt(mean_squared_error(predicted_ratings,test_truth))

2.571962369473256

# ml-1m dataset

In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import math

In [33]:
rs_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('Dataset/ml-1m/ratings.dat', sep='::', names=rs_cols, encoding='latin-1')
ratings_test = pd.read_csv('Dataset/ml-1m/ratings.dat', sep='::', names=rs_cols, encoding='latin-1')

In [34]:
ratings_base.head()

,user_id,movie_id,rating,unix_timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [35]:
n_users_base = ratings_base['user_id'].unique().max()
n_items_base = ratings_base['movie_id'].unique().max()

n_users_base,n_items_base

(6040, 3952)

In [36]:
n_users_test = ratings_test['user_id'].unique().max()
n_items_test = ratings_test['movie_id'].unique().max()
n_users_test,n_items_test

(6040, 3952)

In [37]:
train_matrix = np.zeros((n_users_base, n_items_base))
test_matrix = np.zeros((n_users_test, n_items_test))

In [38]:
for line in ratings_base.itertuples():
    train_matrix[line[1]-1,line[2]-1] = line[3]

In [39]:
test_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
train_matrix

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [41]:
n_users_base, n_items_base

(6040, 3952)

In [42]:
random_users_id_list=[]
while True:
    rnd=np.random.randint(6040)
    if len(random_users_id_list) < 604:
        if rnd in random_users_id_list:
            continue
        else:
            random_users_id_list.append(rnd)
    else:
        break

In [43]:
random_users_id_list.sort()

In [44]:
print(random_users_id_list)

[13, 17, 30, 37, 44, 45, 53, 63, 92, 94, 97, 105, 109, 123, 138, 139, 155, 159, 162, 191, 192, 217, 225, 233, 234, 266, 273, 287, 291, 300, 307, 312, 347, 358, 375, 376, 388, 409, 416, 417, 446, 459, 469, 481, 490, 499, 502, 533, 540, 541, 548, 549, 568, 569, 580, 585, 588, 589, 595, 605, 615, 626, 651, 686, 693, 695, 698, 709, 714, 726, 731, 741, 744, 761, 768, 776, 778, 790, 805, 825, 830, 836, 848, 849, 858, 869, 871, 872, 879, 880, 883, 892, 894, 938, 946, 950, 959, 965, 983, 990, 991, 994, 1009, 1021, 1029, 1038, 1057, 1061, 1063, 1067, 1083, 1111, 1114, 1118, 1129, 1135, 1148, 1158, 1175, 1179, 1185, 1192, 1197, 1199, 1207, 1209, 1241, 1257, 1268, 1278, 1284, 1290, 1305, 1313, 1316, 1329, 1330, 1333, 1342, 1344, 1346, 1383, 1388, 1413, 1421, 1423, 1424, 1439, 1442, 1460, 1461, 1464, 1470, 1473, 1484, 1504, 1507, 1522, 1526, 1537, 1552, 1561, 1571, 1583, 1599, 1604, 1606, 1614, 1617, 1621, 1661, 1671, 1688, 1691, 1709, 1716, 1731, 1739, 1741, 1763, 1788, 1803, 1804, 1831, 1833, 18

In [45]:
users_first_rating_movie=[]

In [46]:
for user in random_users_id_list:
    for item in range(n_items_base):
        if train_matrix[user][item] == 0:
            continue
        else:
            users_first_rating_movie.append(item)
            break

In [47]:
print(random_users_id_list)

[13, 17, 30, 37, 44, 45, 53, 63, 92, 94, 97, 105, 109, 123, 138, 139, 155, 159, 162, 191, 192, 217, 225, 233, 234, 266, 273, 287, 291, 300, 307, 312, 347, 358, 375, 376, 388, 409, 416, 417, 446, 459, 469, 481, 490, 499, 502, 533, 540, 541, 548, 549, 568, 569, 580, 585, 588, 589, 595, 605, 615, 626, 651, 686, 693, 695, 698, 709, 714, 726, 731, 741, 744, 761, 768, 776, 778, 790, 805, 825, 830, 836, 848, 849, 858, 869, 871, 872, 879, 880, 883, 892, 894, 938, 946, 950, 959, 965, 983, 990, 991, 994, 1009, 1021, 1029, 1038, 1057, 1061, 1063, 1067, 1083, 1111, 1114, 1118, 1129, 1135, 1148, 1158, 1175, 1179, 1185, 1192, 1197, 1199, 1207, 1209, 1241, 1257, 1268, 1278, 1284, 1290, 1305, 1313, 1316, 1329, 1330, 1333, 1342, 1344, 1346, 1383, 1388, 1413, 1421, 1423, 1424, 1439, 1442, 1460, 1461, 1464, 1470, 1473, 1484, 1504, 1507, 1522, 1526, 1537, 1552, 1561, 1571, 1583, 1599, 1604, 1606, 1614, 1617, 1621, 1661, 1671, 1688, 1691, 1709, 1716, 1731, 1739, 1741, 1763, 1788, 1803, 1804, 1831, 1833, 18

In [48]:
print(users_first_rating_movie)

[295, 0, 10, 0, 0, 365, 33, 109, 5, 9, 295, 27, 46, 33, 0, 259, 0, 259, 0, 5, 0, 23, 919, 252, 20, 9, 20, 20, 31, 0, 0, 69, 109, 31, 0, 9, 0, 51, 0, 0, 526, 20, 33, 0, 0, 16, 49, 10, 0, 2, 0, 0, 6, 2, 2, 28, 159, 0, 2, 0, 30, 109, 5, 2, 110, 0, 0, 0, 0, 6, 5, 259, 31, 9, 33, 0, 23, 0, 109, 161, 46, 5, 0, 35, 28, 18, 33, 38, 0, 0, 31, 1, 0, 109, 248, 0, 5, 16, 0, 16, 10, 10, 0, 16, 14, 373, 1, 0, 49, 0, 0, 0, 1, 1, 5, 5, 9, 23, 1, 259, 11, 0, 0, 222, 267, 20, 0, 49, 313, 0, 0, 0, 0, 0, 0, 1, 16, 31, 0, 0, 0, 109, 0, 8, 0, 69, 0, 10, 35, 2, 2, 9, 19, 31, 33, 10, 16, 33, 8, 0, 12, 187, 85, 0, 1, 0, 5, 116, 0, 156, 295, 0, 1, 33, 0, 13, 109, 10, 109, 0, 49, 5, 0, 16, 0, 49, 109, 0, 0, 0, 5, 23, 5, 18, 0, 2, 1, 0, 10, 0, 23, 1, 49, 857, 0, 0, 0, 0, 0, 31, 0, 9, 10, 0, 0, 51, 2, 3, 69, 5, 78, 31, 16, 1, 1, 0, 245, 69, 6, 0, 0, 9, 0, 0, 1, 49, 69, 16, 109, 143, 18, 10, 245, 0, 16, 0, 18, 31, 2, 0, 130, 109, 2, 6, 61, 109, 46, 302, 0, 0, 69, 110, 31, 10, 10, 20, 0, 1, 20, 0, 49, 27, 0, 33, 38,

In [49]:
for i in range(94):
    print(train_matrix[random_users_id_list[i]][users_first_rating_movie[i]], end=", ")

5.0, 4.0, 3.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 4.0, 4.0, 5.0, 3.0, 3.0, 2.0, 2.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 3.0, 5.0, 2.0, 5.0, 1.0, 5.0, 3.0, 2.0, 2.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 3.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 3.0, 5.0, 3.0, 4.0, 5.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 5.0, 1.0, 3.0, 4.0, 

In [50]:
for i in range(len(random_users_id_list)):
    test_matrix[random_users_id_list[i]][users_first_rating_movie[i]]=train_matrix[random_users_id_list[i]][users_first_rating_movie[i]]
    train_matrix[random_users_id_list[i]][users_first_rating_movie[i]]=0

In [58]:
print(random_users_id_list)

[13, 17, 30, 37, 44, 45, 53, 63, 92, 94, 97, 105, 109, 123, 138, 139, 155, 159, 162, 191, 192, 217, 225, 233, 234, 266, 273, 287, 291, 300, 307, 312, 347, 358, 375, 376, 388, 409, 416, 417, 446, 459, 469, 481, 490, 499, 502, 533, 540, 541, 548, 549, 568, 569, 580, 585, 588, 589, 595, 605, 615, 626, 651, 686, 693, 695, 698, 709, 714, 726, 731, 741, 744, 761, 768, 776, 778, 790, 805, 825, 830, 836, 848, 849, 858, 869, 871, 872, 879, 880, 883, 892, 894, 938, 946, 950, 959, 965, 983, 990, 991, 994, 1009, 1021, 1029, 1038, 1057, 1061, 1063, 1067, 1083, 1111, 1114, 1118, 1129, 1135, 1148, 1158, 1175, 1179, 1185, 1192, 1197, 1199, 1207, 1209, 1241, 1257, 1268, 1278, 1284, 1290, 1305, 1313, 1316, 1329, 1330, 1333, 1342, 1344, 1346, 1383, 1388, 1413, 1421, 1423, 1424, 1439, 1442, 1460, 1461, 1464, 1470, 1473, 1484, 1504, 1507, 1522, 1526, 1537, 1552, 1561, 1571, 1583, 1599, 1604, 1606, 1614, 1617, 1621, 1661, 1671, 1688, 1691, 1709, 1716, 1731, 1739, 1741, 1763, 1788, 1803, 1804, 1831, 1833, 18

In [59]:
for i in random_users_id_list:
    print(np.sum(test_matrix[i]), end=", ")

5.0, 4.0, 3.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 4.0, 4.0, 5.0, 3.0, 3.0, 2.0, 2.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 3.0, 5.0, 2.0, 5.0, 1.0, 5.0, 3.0, 2.0, 2.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 3.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 3.0, 5.0, 3.0, 4.0, 5.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 5.0, 1.0, 3.0, 4.0, 1.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 4.0, 2.0, 4.0, 1.0, 2.0, 3.0, 4.0, 5.0, 3.0, 3.0, 5.0, 2.0, 3.0, 4.0, 3.0, 2.0, 1.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 2.0, 5.0, 3.0, 5.0, 1.0, 4.0, 2.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 2.0, 2.0, 1.0, 2.0, 3.0, 2.0, 2.0, 4.0, 5.0, 2.0, 4.0, 2.0, 4.0, 3.0, 2.0, 4.0, 4.0, 4.0, 4.0, 2.0, 3.0, 4.0, 2.0, 5.0, 4.0, 4.0, 4.0, 2.0, 4.0, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 4.0, 5.0, 3.0, 3.0, 5.0, 3.0, 4.0, 

In [60]:
for i in range(94):
    print(train_matrix[random_users_id_list[i]][users_first_rating_movie[i]], end=", ")

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [61]:
user_similarity = pairwise_distances(train_matrix, metric='cosine')
user_similarity=1-user_similarity
print('shape: ',user_similarity.shape)
print('type: ',type(user_similarity))
user_similarity

shape:  (6040, 6040)
type:  <class 'numpy.ndarray'>


array([[1.        , 0.09638153, 0.12060981, ..., 0.        , 0.17460369,
        0.13359025],
       [0.09638153, 1.        , 0.1514786 , ..., 0.06611767, 0.0664575 ,
        0.21827563],
       [0.12060981, 0.1514786 , 1.        , ..., 0.12023352, 0.09467506,
        0.13314404],
       ...,
       [0.        , 0.06611767, 0.12023352, ..., 1.        , 0.16171426,
        0.09930008],
       [0.17460369, 0.0664575 , 0.09467506, ..., 0.16171426, 1.        ,
        0.22833237],
       [0.13359025, 0.21827563, 0.13314404, ..., 0.09930008, 0.22833237,
        1.        ]])

In [62]:
for i in range(len(user_similarity)):
    for j in range(len(user_similarity)):
        if i == j:
            user_similarity[i][j]=0
        else:
            continue

In [63]:
for i in range(len(user_similarity)):
    print(user_similarity[i][i], end=" ,")

0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,

0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,

In [64]:
def predict_user_user(train_matrix, user_similarity, n_similar=30):
    similar_n = user_similarity.argsort()[:,n_similar:][:,::-1]
    pred = np.zeros((n_users_base,n_items_base))
    
    for i,users in enumerate(similar_n):
        similar_users_indexes = users
        similarity_n = user_similarity[i,similar_users_indexes]
        matrix_n = train_matrix[similar_users_indexes,:]
        rated_items = similarity_n[:,np.newaxis].T.dot(matrix_n - matrix_n.mean(axis=1)[:,np.newaxis])/ similarity_n.sum()
        pred[i,:]  = rated_items
        
    return pred

In [65]:
predictions = predict_user_user(train_matrix,user_similarity, 30) + train_matrix.mean(axis=1)[:, np.newaxis]
print('predictions shape ',predictions.shape)

predictions shape  (6040, 3952)


In [66]:
predicted_ratings = predictions[test_matrix.nonzero()]
test_truth = test_matrix[test_matrix.nonzero()]

In [67]:
math.sqrt(mean_squared_error(predicted_ratings,test_truth))

2.8785020810298114